In [1]:
import numpy as np

import pandas as pd
import re

In [3]:

class FakeDatabase:
    # return weather based on loc/time (just did something random)
    def get_weather(self, location, weekday):
        location = location.lower()
        weekday = weekday.lower()
        
               
        if weekday == "monday":
            return "rainy"
        if weekday == "tuesday":
            return "cloudy"
        if weekday == "wednesday":
            return "cloudy with rain"
        if weekday == "thursday":
            return "windy"
        if weekday == "friday":
            return "sparsely cloudy with some sunshine"
        if weekday == "saturday":
            return "very rainy"
        if weekday == "sunday":
            return "sunny"
        

    
    # return restaurant based on loc/time
    def get_restaurant(self, price, food, area):
        if food.split()[0] == "thai":
            return {"name": "Thai Food House", "address": "Eklandagatan 76"}
    
        if food.split()[0] == "italian":
            return {"name": "La Cucina Italiana", "address": "Gibraltargatan 98"}
        
        if food.split()[0] == "mexican":
            return {"name": "Fonda doña Lupita", "address": "Kungsgatan 20"}
        
        if food.split()[0] == "greek":
            return {"name": "Agios Aggelios", "address": "Medicinaregatan 98"}
        
        if food.split()[0] == "chinese":
            return {"name": "Noodle house", "address": "Avenyn 1200"}
    
    
    # return next bus/tram based on origin/destination/time
    def get_transport(self, origin, destination, time):
        return {"type": "bus", "time": "12:35 PM"}
    

In [4]:
class Frame:
    def __init__(self, database, domain, intent):
        self.database = database
        self.domain = domain
        self.intent = intent
        
    def fill_slots():
        pass
    
    def prompt():
        pass
        
class WeatherFrame(Frame):
    def __init__(self, database):
        super().__init__(database, domain = "WEATHER-FORECAST", intent = "WEATHER")
        self.location = None
        self.weekday = None
        
    def fill_slots(self, user_input):
        if not self.location:
            location_regex = "in (stockholm|gothenburg|san francisco|new york|berlin|london)"
            m = re.search(location_regex, user_input)
            if m:
                # if the city has two words
                city = m.group(0).split()[1:]
                city = ' '.join(city)
#                 self.location = m.group(0).split()[1].capitalize()
                self.location = city.capitalize()
                
        if not self.weekday:
            weekday_regex = "on (monday|tuesday|wednesday|thursday|friday|saturday|sunday)"
            m = re.search(weekday_regex, user_input)
            if m:
                self.weekday = m.group(0).split()[1].capitalize()

    def prompt_location(self):
        return "In which city would you like a weather forecast?"
    
    def prompt_weekday(self):
        if self.location:
            return "On what day are you interested in receiving a weather forecast in {}?".format(self.location)
        else:
            return "On what day are you interested in receiving a weather forecast?"
    
    def prompt(self):
        if not self.location:
            return self.prompt_location()
        
        if not self.weekday:
            return self.prompt_weekday()
        
        # both have been filled, return answer
        weather = self.database.get_weather(self.location, self.weekday)
        return "The weather in {} on {} is: {}.".format(self.location, self.weekday, weather)
    
    def is_filled(self):
        return self.location and self.weekday
        
class RestaurantFrame(Frame):
    def __init__(self, database):
        super().__init__(database, domain = "RESTAURANT", intent = "FIND-RESTAURANT")
        self.price = None
        self.food = None
        self.location = None
        
    def fill_slots(self, user_input):
        if not self.price:
            price_regex = "cheap|expensive|moderate"
            m = re.search(price_regex, user_input)
            if m:
                self.price = m.group(0).capitalize()
                
        if not self.food:
            food_regex = "(italian|thai|chinese|greek|mexican) food"
            m = re.search(food_regex, user_input)
            if m:
                self.food = m.group(0)

        if not self.location:
            location_regex = "(in|near|around) (downtown|the south|the north|the east|the west)"
            m = re.search(location_regex, user_input)
            if m:
                self.location = m.group(0)
                

    def prompt_price(self):
        return "What price range are you interested in?"
        
    def prompt_food(self):
        return "What sort of food are you looking for?"
    
    def prompt_location(self):
        return "What area would you like to eat at?"
    
    def prompt(self):
        if not self.food:
            return self.prompt_food()
        
        if not self.location:
            return self.prompt_location()
        
        if not self.price:
            return self.prompt_price()
        
        
        
        # both have been filled, return answer
        restaurant = self.database.get_restaurant(self.price, self.food, self.location)
        ret = "{} {} (located {}) can be found at the restaurant '{}'. The address is {}.".format(self.price, self.food, self.location, restaurant["name"], restaurant["address"])
        return ret

    
    def is_filled(self):
        return self.price and self.food and self.location
        
class TransportFrame(Frame):
    def __init__(self, database):
        super().__init__(database, domain = "TRANSPORT", intent = "FIND-TRAM-OR-BUS")
        self.origin = None
        self.destination = None
        self.time = None
        
    def fill_slots(self, user_input):
        if not self.origin:
            origin_regex = "from [a-zA-Z]+"
            m = re.search(origin_regex, user_input)
            if m:
                self.origin = m.group(0).split()[1].capitalize()
                
        if not self.destination:
            destination_regex = "to [a-zA-Z]+"
            m = re.search(destination_regex, user_input)
            if m:
                self.destination = m.group(0).split()[1].capitalize()
                
        if not self.time:
            time_regex = "at (1[0-2]|0?[1-9])(:[0-5][0-9])? (am|pm)"
            m = re.search(time_regex, user_input)
            if m:
                self.time = m.group(0)

    def prompt_origin(self):
        return "Where are you leaving from?"
    
    def prompt_destination(self):
        return "Where would you like to go?"
    
    def prompt_time(self):
        return "At what time would you like to depart?"
    
    def prompt(self):
        if not self.destination:
            return self.prompt_destination()
        
        if not self.origin:
            return self.prompt_origin()
        
        if not self.time:
            return self.prompt_time()
        
        # both have been filled, return answer
        transport = self.database.get_transport(self.origin, self.destination, self.time)
        return "There is a {} going from {} to {} at {}.".format(transport["type"], self.origin, self.destination, transport["time"])
    
    def is_filled(self):
        return self.origin and self.destination and self.time
        

In [5]:
import re
class DigitalAssistant:
    WEATHER = 1
    RESTAURANT = 2
    TRANSPORT = 3
    
    WEATHER_KEYWORDS = ["weather", "rainy", "sunny"]
    RESTAURANT_KEYWORDS = ["restaurant", "hungry", "food", "eat"]
    TRANSPORT_KEYWORDS = ["transport", "bus", "tram", "travel"]
    
    def __init__(self):
        self.db = FakeDatabase()
        self.weather_frame = None
        self.restaurant_frame = None
        self.transport_frame = None
        self.current_intent = None
        
    def say(self, sentence):
        print("ASSISTANT: ", sentence)
        
    def identify_intent(self, user_input):
        if any(keyword in user_input for keyword in self.WEATHER_KEYWORDS):
            self.current_intent = self.WEATHER
            if not self.weather_frame:
                self.weather_frame = WeatherFrame(self.db)
        elif any(keyword in user_input for keyword in self.RESTAURANT_KEYWORDS):
            self.current_intent = self.RESTAURANT
            if not self.restaurant_frame:
                self.restaurant_frame = RestaurantFrame(self.db)
        elif any(keyword in user_input for keyword in self.TRANSPORT_KEYWORDS):
            self.current_intent = self.TRANSPORT
            if not self.transport_frame:
                self.transport_frame = TransportFrame(self.db)
        #else:
            #intent unchanged or unkown
            
        # debug    
       # if self.current_intent == self.WEATHER:
       #     print("CURRENT INTENT: ", "WEATHER")
       # elif self.current_intent == self.RESTAURANT:
       #     print("CURRENT INTENT: ", "RESTAURANT")
       # elif self.current_intent == self.TRANSPORT:
       #     print("CURRENT INTENT: ", "TRANSPORT")
            
    def process_input(self, user_input):
        #user_input_split = user_input.lower().split()
        self.identify_intent(user_input)
        
        if self.current_intent == DigitalAssistant.WEATHER:
            self.weather_frame.fill_slots(user_input)
            self.say(self.weather_frame.prompt())
            if self.weather_frame.is_filled():
                self.weather_frame = None
                self.say("What else can I do for you?")
        elif self.current_intent == DigitalAssistant.RESTAURANT:
            self.restaurant_frame.fill_slots(user_input)
            self.say(self.restaurant_frame.prompt())
            if self.restaurant_frame.is_filled():
                self.restaurant_frame = None
                self.say("What else can I do for you?")
        elif self.current_intent == DigitalAssistant.TRANSPORT:
            self.transport_frame.fill_slots(user_input)
            self.say(self.transport_frame.prompt())
            if self.transport_frame.is_filled():
                self.transport_frame = None
                self.say("What else can I do for you?")
        else:
            self.say("Interesting, what can I do for you?")
        

In [7]:
assistant = DigitalAssistant()
assistant.say("Hi, what can I do for you?")
while(True):
    user_input = input()
    #print(user_input)
    assistant.process_input(user_input.lower())

ASSISTANT:  Hi, what can I do for you?


 I would like to eat something


ASSISTANT:  What sort of food are you looking for?


 mexican food


ASSISTANT:  What area would you like to eat at?


...

ASSISTANT:  What price range are you interested in?


 expensive


ASSISTANT:  Expensive mexican food (located in the east) can be found at the restaurant 'Fonda doña Lupita'. The address is Kungsgatan 20.
ASSISTANT:  What else can I do for you?


 ^C


AttributeError: 'NoneType' object has no attribute 'fill_slots'